In [1]:
import pandas as pd
import numpy as np
import random as rng

from schedule import Schedule
from paintshop import PaintShop
from move import Move


# DEBUG (reload import)
from importlib import reload
import paintshop
import move
reload(paintshop)
reload(move)
from paintshop import PaintShop
from move import Move

       0   1   2   3   4   5   6   7   8   9  10   8777.53
M1: [ 18  14   2  28  10  13  12  24  23  29     ] 2575.00 (29%)
M2: [ 21   7  16   3   5  17   1   6  26         ] 2101.20 (24%)
M3: [ 11  27   8   4  22  15   0  19   9  20  25 ] 4101.33 (47%)


# Settings

In [2]:
SEED = 420

# Setup

In [3]:
rng.seed(SEED)
PS = PaintShop()

# Classes

Metaheuristics:
- Multistart:  Improving multiple starts and take the best local optimum
- Taboo Search: Reverse steps are forbidden (taboo-list). Tabu moves are removed from the list after a number of moves. Stop at a certain number of iterations OR when all available moves are taboo. Keep track of incumbent solution.
- Simulated Annealing: Temperature & cooling schedule. Allways accepts improving moves. Non-improving moves with probablity based on the obj. improvement and temperature. Randomly choose a move, compute the gain. If it improves, accept, otherwise accept with probability e^(delta_obj / q). Update incumbent solution. Reduce temperature q. See graph in slides.
- Genetic algorithms: Many different ideas. Population of solutions.
   - Start with a number of random solutions.
   - Create new solutions by combining pairs.
   - Mutations sometimes.
   - Select survivors (elite).



# Solution

In [4]:
def heuristic_constructive_simple() -> Schedule:
    """Constructs a solution according to the following heuristic:
    1. Create an empty solution. (Empty lists (representing order-numbers) by machine-numbers in a dictionary)
    2. Assign the order with the lowest order-number to the machine with the lowest amount of assigned orders, adding it to the end to the order-queue for that machine. The tiebreaking rule is that the machine with the lowest machine-number gets the order.
    3. Go to step 2 unless all orders are assigned.
    
    Returns:
        dict[int, list[int]]: The constructed solution. 
    """
    
    
    # Construct an empty solution dictionary.
    schedule = Schedule()
    
    # For each order (ordered by order-number ascending).
    for order_id in PS.order_ids:
        
        # Determine machine index with the shortest queue (adding a machine's index (scaled to a fraction) works like the tiebreaking rule).
        machine_id_next = sorted(
            PS.machine_ids, 
            key = lambda i:
                len(schedule[i, :]) +
                i / len(PS.machine_ids)
        )[0]
        
        # Add order to machine queue.
        schedule[machine_id_next, :] += [order_id]
        
    return schedule

In [5]:
def heuristic_constructive_simple_2() -> Schedule:
    """Constructs a solution according to the following heuristic:
    1. Create an empty solution. (Empty lists (representing order-numbers) by machine-numbers in a dictionary)
    2. Assign the order with the lowest order-number to the machine with the lowest amount of assigned orders, adding it to the end to the order-queue for that machine. The tiebreaking rule is that the machine with the lowest machine-number gets the order.
    3. Go to step 2 unless all orders are assigned.
    
    Returns:
        dict[int, list[int]]: The constructed solution. 
    """
    
    
    # Construct an empty solution dictionary.
    schedule = Schedule()
    
    # For each order (ordered by order-number ascending).
    for order_id in PS.order_ids:
        
        # Determine machine index with the shortest queue (adding a machine's index (scaled to a fraction) works like the tiebreaking rule).
        machine_id_next = sorted(
            PS.machine_ids, 
            key = lambda i:
                schedule.get_finish_time(i) + 
                i / len(PS.machine_ids), 
        )[0]
        
        # Add order to machine queue.
        schedule[machine_id_next, :] += [order_id]
        
    return schedule

In [6]:

def heuristic_constructive_simple_3() -> Schedule:
    """Constructs a solution according to the following heuristic:
    1. Create an empty solution. (Empty lists (representing order-numbers) by machine-numbers in a dictionary)
    2. Assign the order with the lowest order-number to the machine with the lowest amount of assigned orders, adding it to the end to the order-queue for that machine. The tiebreaking rule is that the machine with the lowest machine-number gets the order.
    3. Go to step 2 unless all orders are assigned.
    
    Returns:
        dict[int, list[int]]: The constructed solution. 
    """
    
    
    # Construct an empty solution dictionary.
    schedule = Schedule()
    
    # For each order (ordered by order-number ascending).
    for order_id in sorted(PS.order_ids, key = lambda order_id: PS.orders.loc[order_id, 'deadline']):
        
        # Determine machine index with the shortest queue (adding a machine's index (scaled to a fraction) works like the tiebreaking rule).
        machine_id_next = sorted(
            PS.machine_ids, 
            key = lambda i:
                schedule.get_finish_time(i) + 
                i / len(PS.machine_ids), 
        )[0]
        
        # Add order to machine queue.
        schedule[machine_id_next, :] += [order_id]
        
    return schedule

In [7]:

def heuristic_constructive_random() -> Schedule:
    """Constructs a solution according to the following heuristic:
    1. Create an empty solution. (Empty lists (representing order-numbers) by machine-numbers in a dictionary)
    2. Assign the order with the lowest order-number to the machine with the lowest amount of assigned orders, adding it to the end to the order-queue for that machine. The tiebreaking rule is that the machine with the lowest machine-number gets the order.
    3. Go to step 2 unless all orders are assigned.
    
    Returns:
        dict[int, list[int]]: The constructed solution. 
    """
    
    
    # Construct an empty solution dictionary.
    schedule = Schedule()
    
    # Create list of shuffled order ID's
    order_ids_remaining = PS.order_ids
    rng.shuffle(order_ids_remaining)
    
    while len(order_ids_remaining) > 0:
        
        next_order_id_index = rng.choice(range(len(order_ids_remaining)))
        
        schedule[rng.choice(PS.machine_ids), :] += [order_ids_remaining[next_order_id_index]]
        
        order_ids_remaining = np.delete(order_ids_remaining, next_order_id_index)     
         
    return schedule

In [8]:
import math


def heuristic_improvement_best(initial: Schedule) -> Schedule:
    
    current_schedule = initial
    
    print(f"Initial:")
    print(str(current_schedule) + "\n")
    
    while True:
    
        best_swap = None
        best_swap_cost = initial.get_cost()
        
        for swap in current_schedule.get_swaps():
            swapped_schedule: Schedule = current_schedule.get_copy(swap)
            swapped_cost = swapped_schedule.get_cost()
            
            # print(f"Swap: {swap}, Cost: {swapped_cost}")
            
            if (swapped_cost < best_swap_cost):
                best_swap = swap
                best_swap_cost = swapped_cost
        
        # Break the loop if no improving swap was found.
        if best_swap == None:
            break
        
        current_schedule.swap(best_swap)
        print(f"Swapped: {best_swap}.")
        print(str(current_schedule) + "\n")
        
    return current_schedule

In [9]:
def heuristic_improvement_first(initial: Schedule) -> Schedule:
    
    current_schedule = initial
    
    print(f"Initial:")
    print(str(current_schedule) + "\n")
    
    while True:
    
        first_improvement = None
        initial_swap_cost = initial.get_cost()
        
        for swap in current_schedule.get_swaps():
            swapped_schedule: Schedule = current_schedule.get_copy(swap)
            swapped_cost = swapped_schedule.get_cost()
            
            # print(f"Swap: {swap}, Cost: {swapped_cost}")
            # 
            if (swapped_cost < initial_swap_cost):
                first_improvement = swap
                break
        
        # Break the loop if no improving swap was found.
        if first_improvement == None:
            break
        
        current_schedule.swap(first_improvement)
        print(f"Swapped: {first_improvement}.")
        print(str(current_schedule) + "\n")
        
    return current_schedule

In [10]:
# heuristic_improvement_best(heuristic_constructive_simple()).draw()
# heuristic_constructive_simple().draw()

In [11]:
print('\n\n'.join([
    f"{schedule}" for heuristic, schedule in {
        "constructive_simple": heuristic_constructive_simple(),
        "constructive_simple_2": heuristic_constructive_simple_2(),
        "constructive_simple_3": heuristic_constructive_simple_3(),
        "constructive_random": heuristic_constructive_random()
    }.items()
]))

       0   1   2   3   4   5   6   7   8   9   1414.11
M1: [  0   3   6   9  12  15  18  21  24  27 ] 565.00 (40%)
M2: [  1   4   7  10  13  16  19  22  25  28 ] 222.00 (16%)
M3: [  2   5   8  11  14  17  20  23  26  29 ] 627.11 (44%)

       0   1   2   3   4   5   6   7   8   9  10  11   2815.77
M1: [  0   3   7  12  14  17  20  22  25  29         ] 760.50  (27%)
M2: [  1   4   6   9  10  13  15  19  21  24  26  28 ] 1440.60 (51%)
M3: [  2   5   8  11  16  18  23  27                 ] 614.67  (22%)

       0   1   2   3   4   5   6   7   8   9  10   4181.39
M1: [  2  10   6   7  13  19  18  16  28  23     ] 1632.50 (39%)
M2: [  3  11   8   4  12  15  14  26  25  21  29 ] 1445.00 (35%)
M3: [  0   1   5   9  24  20  17  27  22         ] 1103.89 (26%)

       0   1   2   3   4   5   6   7   8   9  10  11  12   8345.60
M1: [  3  12  23  21   7  22   0  16  19   9   8         ] 3221.00 (39%)
M2: [  6  26   5  20  18  17   2  14  11  29  15  10  24 ] 4484.60 (54%)
M3: [ 28  27   4  25   1 

### Move
This abstract base class (ABC) and it's subclasses handle the move operations. The static method 'get_moves' of 'Move' can be called to get a list of every possible move on a given schedule. The subclasses handle the different operations involved. The 'Move' class is the interface.

# Code Validation

In [12]:
import itertools as iter

In [13]:
# Ensure all moves produce a schedule that is different from the one we started with.
s = heuristic_constructive_random()
print(s)

moves: list[Move] = Move.get_moves(s)
print(f'\nAmount of moves: {len(moves)}')

moved_schedules = [move.get_moved(s) for move in moves]
print(f"\nMoves that do nothing: {sum([(s == moved_s) for moved_s in moved_schedules])}")

print(f'\nUnique schedules after moves: {len(set(moved_schedules))}')

# Determine moves that produce the identical schedule (sorting by cost is not guaranteed to work)
sorted_moves = sorted(moves, key = lambda move: move.get_moved(s).get_cost())
for schedule, group in iter.groupby(sorted_moves, lambda move: move.get_moved(s)):
    if (len(list(group)) > 1):
        print(schedule)
        for move in group:
            print(move)

print(f'\nMoves that produce the same schedules:')

       0   1   2   3   4   5   6   7   8   9  10   9301.79
M1: [ 26  13  11   1  27  25  24   2  20   4     ] 3434.50 (37%)
M2: [ 15  28   3  23  17  22   9  16  10         ] 1865.40 (20%)
M3: [ 19  29   6  21  12   5  14   7  18   8   0 ] 4001.89 (43%)

Amount of moves: 927

Moves that do nothing: 0

Unique schedules after moves: 900
       0   1   2   3   4   5   6   7   8   9  10   8445.19
M1: [ 26  13  11   1  27  25  24   2  20   4     ] 3434.50 (41%)
M2: [ 15   3  28  23  17  22   9  16  10         ] 1008.80 (12%)
M3: [ 19  29   6  21  12   5  14   7  18   8   0 ] 4001.89 (47%)
       0   1   2   3   4   5   6   7   8   9  10   8798.46
M1: [ 26  13  11   1  27  25  24   2  20   4     ] 3434.50 (39%)
M2: [ 15  28   3  23  17  22   9  16  10         ] 1865.40 (21%)
M3: [ 19  29   6  21  12   5   7  14  18   8   0 ] 3498.56 (40%)
       0   1   2   3   4   5   6   7   8   9  10   8825.19
M1: [ 26  13  11   1  27  25  24   2  20   4     ] 3434.50 (39%)
M2: [ 28  15   3  23  17  22   

In [14]:
self = heuristic_constructive_random()

queue_lengths = [len(queue) for queue in self.queues]

queue_columns = [
    max([
        len(str(self[mi, oi])) for mi in PS.machine_ids if oi < len(self[mi, :])
    ]) for oi in range(max(queue_lengths))
]
print(queue_columns)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2]


In [15]:
print("white\x1b[31mred\x1b[0mwhite")

whiteredwhite
